In [5]:
import chess.pgn
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

In [6]:


pgn_file = "/Users/ruvinjagoda/Desktop/Aka/AIP/Chess_code/2023_full.pgn"
# Function to parse PGN file and yield game data in batches
def parse_pgn_file_in_batches(file_path, batch_size=5000):
    with open(file_path) as f:
        while True:
            games = []
            for _ in range(batch_size):
                game = chess.pgn.read_game(f)
                if game is None:
                    break
                games.append(game)
            if not games:
                break
            yield games



In [8]:
import os
import chess.pgn

# Function to filter games based on average Elo
def filter_games_by_average_elo(games, min_average_elo):
    filtered_games = []
    for game in games:
        white_elo = int(game.headers.get('WhiteElo', 0))
        black_elo = int(game.headers.get('BlackElo', 0))
        if (white_elo + black_elo) / 2 >= min_average_elo:
            filtered_games.append(game)
    return filtered_games

# Function to save filtered games to a new PGN file
def save_games_to_pgn(filtered_games, output_file_path):
    with open(output_file_path, 'w') as f:
        for game in filtered_games:
            f.write(str(game) + '\n\n')

# Function to load games from a PGN file
def load_games_from_pgn_file(file_path):
    games = []
    with open(file_path) as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break
            games.append(game)
    return games

# Path to the folder containing the PGN files
input_folder = "/Users/ruvinjagoda/Desktop/Aka/AIP/Database/Standerd_above_2000 games/"
output_file_path = "/Users/ruvinjagoda/Desktop/Aka/AIP/Database/high_elo_games.pgn"

# Minimum average Elo for filtering
min_average_elo = 2500

# Collect all filtered games
all_filtered_games = []

# Iterate through all PGN files in the specified folder
for filename in os.listdir(input_folder):
    if filename.endswith(".pgn"):
        file_path = os.path.join(input_folder, filename)
        print(f"Processing file: {file_path}")
        games = load_games_from_pgn_file(file_path)
        filtered_games = filter_games_by_average_elo(games, min_average_elo)
        all_filtered_games.extend(filtered_games)

# Print count of filtered games
print("Number of filtered games:", len(all_filtered_games))

# Save all filtered games to a new PGN file
save_games_to_pgn(all_filtered_games, output_file_path)

print("Filtered games saved to:", output_file_path)


Processing file: /Users/ruvinjagoda/Desktop/Aka/AIP/Database/Standerd_above_2000 games/ficsgamesdb_2018_standard2000_movetimes_390458.pgn
Processing file: /Users/ruvinjagoda/Desktop/Aka/AIP/Database/Standerd_above_2000 games/ficsgamesdb_2023_standard2000_movetimes_390449.pgn
Processing file: /Users/ruvinjagoda/Desktop/Aka/AIP/Database/Standerd_above_2000 games/ficsgamesdb_2017_standard2000_movetimes_390459.pgn
Processing file: /Users/ruvinjagoda/Desktop/Aka/AIP/Database/Standerd_above_2000 games/ficsgamesdb_2015_standard2000_movetimes_390461.pgn
Processing file: /Users/ruvinjagoda/Desktop/Aka/AIP/Database/Standerd_above_2000 games/ficsgamesdb_2016_standard2000_movetimes_390460.pgn
Processing file: /Users/ruvinjagoda/Desktop/Aka/AIP/Database/Standerd_above_2000 games/ficsgamesdb_2014_standard2000_movetimes_390462.pgn
Processing file: /Users/ruvinjagoda/Desktop/Aka/AIP/Database/Standerd_above_2000 games/ficsgamesdb_2010_standard2000_movetimes_390466.pgn
Processing file: /Users/ruvinjagod

OSError: [Errno 28] No space left on device

In [13]:
import numpy as np
import chess
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Adjust the dimensions of the shortcut to match the output
    shortcut = layers.Conv2D(filters, (1, 1), strides=stride, padding='same')(shortcut)
    shortcut = layers.BatchNormalization()(shortcut)

    x = layers.add([shortcut, x])
    x = layers.Activation('relu')(x)
    return x

def create_advanced_model():
    inputs = Input(shape=(8, 8, 12))

    # Initial Convolution and Pooling
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.3)(x)

    # Residual Block 1
    x = residual_block(x, 128)
    x = residual_block(x, 128)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.4)(x)

    # Residual Block 2
    x = residual_block(x, 256)
    x = residual_block(x, 256)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.2)(x)

    # Fully Connected Layers
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    # Output Layer
    outputs = layers.Dense(4096, activation='softmax')(x)

    # Model Definition
    model = models.Model(inputs=inputs, outputs=outputs)

    # Compile Model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Other functions remain the same
def board_to_one_hot(board):
    one_hot = np.zeros((8, 8, 12), dtype=np.int8)
    piece_map = {chess.PAWN: 0, chess.KNIGHT: 1, chess.BISHOP: 2,
                 chess.ROOK: 3, chess.QUEEN: 4, chess.KING: 5}
    for square in chess.scan_reversed(chess.BB_ALL):
        piece = board.piece_at(square)
        if piece is not None:
            piece_index = piece_map[piece.piece_type] + (6 if piece.color else 0)
            one_hot[chess.square_rank(square), chess.square_file(square), piece_index] = 1
    return one_hot

def move_to_label(move):
    from_square = move.from_square
    to_square = move.to_square
    return from_square * 64 + to_square

def data_generator(games, batch_size):
    X_batch, y_batch = [], []
    while True:
        for game in games:
            board = game.board()
            for move in game.mainline_moves():
                X_batch.append(board_to_one_hot(board))
                y_batch.append(move_to_label(move))
                board.push(move)
                if len(X_batch) == batch_size:
                    yield np.array(X_batch, dtype=np.float32), to_categorical(np.array(y_batch), num_classes=4096)
                    X_batch, y_batch = [], []

class PrintAccuracyCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch + 1} - Training Accuracy: {logs['accuracy']:.4f}, Validation Accuracy: {logs['val_accuracy']:.4f}")

# Training parameters
batch_size = 128
games_per_batch = 5000
epochs_per_cycle = 5  # Number of epochs to train on each batch of games

# Train the model in cycles
total_games = 100000
model = create_advanced_model()

for cycle in range(total_games // games_per_batch):
    print(f"Training cycle {cycle + 1}/{total_games // games_per_batch}")
    train_games = next(parse_pgn_file_in_batches(pgn_file, games_per_batch))
    steps_per_epoch = sum(1 for game in train_games for _ in game.mainline_moves()) // batch_size
    
    history = model.fit(
        data_generator(train_games, batch_size),
        steps_per_epoch=steps_per_epoch,
        epochs=epochs_per_cycle,
        validation_data=data_generator(train_games, batch_size),  # Using train data for validation just for simplicity
        validation_steps=steps_per_epoch // 10  # Adjust validation steps accordingly
    )
    
    print(f"Cycle {cycle + 1} Training accuracy: {history.history['accuracy'][-1]}")
    print(f"Cycle {cycle + 1} Validation accuracy: {history.history['val_accuracy'][-1]}")

# Evaluate the model using the last chunk of test games
test_games = next(parse_pgn_file_in_batches(pgn_file, games_per_batch))
validation_steps = sum(1 for game in test_games for _ in game.mainline_moves()) // batch_size

test_loss, test_acc = model.evaluate(data_generator(test_games, batch_size), steps=validation_steps)
print(f"Test accuracy: {test_acc}")


Training cycle 1/20
Epoch 1/5


2024-05-24 17:31:07.179734: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


2903/2903 [==============================] - ETA: 0s - loss: 6.9022 - accuracy: 0.0219

2024-05-24 17:56:33.814675: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


2903/2903 [==============================] - 1551s 534ms/step - loss: 6.9022 - accuracy: 0.0219 - val_loss: 6.3611 - val_accuracy: 0.0400
Epoch 2/5
2903/2903 [==============================] - 1471s 507ms/step - loss: 6.2953 - accuracy: 0.0413 - val_loss: 6.0183 - val_accuracy: 0.0565
Epoch 3/5
2903/2903 [==============================] - 1488s 513ms/step - loss: 6.0190 - accuracy: 0.0595 - val_loss: 5.7706 - val_accuracy: 0.0671
Epoch 4/5
2903/2903 [==============================] - 1714s 590ms/step - loss: 5.7513 - accuracy: 0.0786 - val_loss: 5.4884 - val_accuracy: 0.0822
Epoch 5/5
2855/2903 [============================>.] - ETA: 27s - loss: 5.5477 - accuracy: 0.0943

KeyboardInterrupt: 

In [ ]:
model.save("/kaggle/working/model_weights_v3.h5")

In [8]:
# Function to create or load a model
def create_model(weights_path=None):
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 12), padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.1),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.2),

        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.05),

        layers.Flatten(),
        layers.Dense(1024, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.1),

        layers.Dense(4096, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    if weights_path:
        model.load_weights(weights_path)

    return model

# Define the path where model weights will be saved






model = create_model("./model_weights_v91.h5")



# Function to check if a move is legal
def is_legal(board, move):
    return move in board.legal_moves

# Function to analyze model predictions and identify illegal moves
def analyze_illegal_moves(model, games, batch_size):
    illegal_moves_count = 0
    total_moves_count = 0
    illegal_move_reasons = []

    for game in games:
        board = game.board()
        for move in game.mainline_moves():
            total_moves_count += 1
            input_board = np.expand_dims(board_to_one_hot(board), axis=0)
            prediction = model.predict(input_board)
            predicted_move_index = np.argmax(prediction)
            predicted_move = chess.Move(from_square=predicted_move_index // 64, to_square=predicted_move_index % 64)
            if not is_legal(board, predicted_move):
                illegal_moves_count += 1
                illegal_move_reasons.append((board.fen(), predicted_move, move))
            board.push(move)

    return illegal_moves_count, total_moves_count, illegal_move_reasons


# Analyze illegal moves in test games
illegal_moves, total_moves,illegal_move_reasons = analyze_illegal_moves(model, test_games[:1], batch_size)

print(f"Total moves: {total_moves}")
print(f"Illegal moves: {illegal_moves}")
print(f"Percentage of illegal moves: {(illegal_moves / total_moves) * 100:.2f}%")

# Print reasons for illegal moves
print("Reasons for illegal moves:")
for fen, predicted_move, actual_move in illegal_move_reasons:
    print(f"FEN: {fen}")
    print(f"Predicted Move: {predicted_move.uci()}")
    print(f"Actual Move: {actual_move.uci()}")
    print()

NameError: name 'test_games' is not defined